## Importing Required Packages

In [1]:
import os
import numpy as np
import pandas as pd
import statistics
import rioxarray
import xarray as xr
import rasterio
import statistics
from tqdm import tqdm

## Function to convert data to annualy

In [2]:
def aggregate_to_annual(data_array):
    """
    Aggregate monthly data to annual values.
    """
    return data_array.resample(time='A').mean(dim='time')

## Function to calculate max and min value per cell
Also also applying first filter

In [3]:
def calculate_max_min_per_cell(data_array):
    """
    Calculate the maximum and minimum values excluding outliers
    """
    mean_val = data_array.mean(dim='time')
    std_val = data_array.std(dim='time')
    
    # Exclude outliers within each grid cell
    valid_data = data_array.where((data_array >= (mean_val - 3 * std_val)) & (data_array <= (mean_val + 3 * std_val)), other=np.nan)
    
    # Calculate the maximum and minimum values for each cell after excluding outliers
    max_val_per_cell = valid_data.max(dim='time')
    min_val_per_cell = valid_data.min(dim='time')
    
    return max_val_per_cell, min_val_per_cell

## Load Data

In [4]:
# Define paths to your data
data_dir = "../Processed_Data/HTI_Indices_Resampled/"
koppen_map_path = "../Raw_Data/koppen_geiger_0p5.tif"
output_path = "../Processed Data/Output of analysis"

# Load Koppen map
koppen_data = rioxarray.open_rasterio(koppen_map_path).squeeze()
regions = np.unique(koppen_data)

# Models and indices
models = os.listdir(data_dir)
scenarios = ['SSP585']
indices = ["AT_in", "AT_out", "DI", "ET", "HI", "HMI", "MDI", "NET", "SAT", "sWGBT", "WBT", "WCT"]  # Replace with actual indices names

max_min_results = {region: {index: {model: {"max": None, "min": None, "max_cross_consi": None, "min_cross_consi": None, "max_cross_years": [], "min_cross_years": []} for model in models} for index in indices}  for region in regions}

## Finding Max and Min Values of Indices on historical Period (1850-2014)

In [5]:
for model in models:
    print(f'Precssing of {model} is started')
    model_dir = os.path.join(data_dir, model)
    model_dir_historical = os.path.join(data_dir, model, 'Historical')
    
    for index in tqdm(indices,"processing Indices"):
        
        index_file = os.path.join(model_dir_historical, f'resampled_{index}.nc')
        data_array_historical = xr.open_dataset(index_file)[index]
        data_array_annual_historical = aggregate_to_annual(data_array_historical)
        
        max_val, min_val = calculate_max_min_per_cell(data_array_annual_historical)  # got historical max and min value per cell of index
        
        for region in regions:
            region_mask = (koppen_data == region).compute()
            
            region_max_val_per_cell = max_val.where(region_mask, other = np.nan)
            region_min_val_per_cell = min_val.where(region_mask, other = np.nan)
            
            region_mean_of_max_val_per_cell = region_max_val_per_cell.mean()
            region_mean_of_min_val_per_cell = region_min_val_per_cell.mean()
            
            region_std_of_max_val_per_cell = region_max_val_per_cell.std()
            region_std_of_min_val_per_cell = region_min_val_per_cell.std()
          
            Valid_max_val_for_region = region_max_val_per_cell.where((region_max_val_per_cell >= (region_mean_of_max_val_per_cell-3*region_std_of_max_val_per_cell)) & (region_max_val_per_cell <= (region_mean_of_max_val_per_cell + 3*region_std_of_max_val_per_cell)) , other = np.nan)
            
            Valid_min_val_for_region = region_min_val_per_cell.where((region_min_val_per_cell >= (region_mean_of_min_val_per_cell-3*region_std_of_min_val_per_cell)) & (region_min_val_per_cell <= (region_mean_of_min_val_per_cell + 3*region_std_of_min_val_per_cell)) , other = np.nan)
            
            max_val_of_index_for_this_region = Valid_max_val_for_region.max()
            min_val_of_index_for_this_region = Valid_min_val_for_region.min()
            
            max_min_results[region][index][model]["max"] = max_val_of_index_for_this_region
            max_min_results[region][index][model]["min"] = min_val_of_index_for_this_region
            
    print(f'Processing of {model} is Done\n')

Precssing of IIT-ESM (INDIA) is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [01:33<00:00,  7.76s/it]

Processing of IIT-ESM (INDIA) is Done



## Finding Exposure Timing

In [6]:
for model in models:
    print(f'Precssing of {model} is started')
    model_dir = os.path.join(data_dir, 'IIT-ESM (INDIA)')
    
    for scenario in scenarios:
        scenario_dir = os.path.join(model_dir,scenario)

        for index in tqdm(indices,"processing Indices"):
            index_file_of_scenario = os.path.join(scenario_dir,f'resampled_{index}.nc')
            data_array_scenario = xr.open_dataset(index_file_of_scenario)[index]
            data_array_annual_scenario = aggregate_to_annual(data_array_scenario)
            
            #filtering######################
            mean_val = data_array_annual_scenario.mean(dim='time')
            std_val = data_array_annual_scenario.std(dim='time')

            # Exclude outliers within each grid cell
            valid_data = data_array_annual_scenario.where((data_array_annual_scenario >= (mean_val - 3 * std_val)) & (data_array_annual_scenario <= (mean_val + 3 * std_val)), other=np.nan)
            data_array_annual_scenario = valid_data
            ################################
            
            filtered_scenario_data = data_array_annual_scenario  # for now assuming filter is applied

            for region in regions:

                region_mask = (koppen_data == region).compute()
                
                region_data = filtered_scenario_data.where(region_mask, other = np.nan)

                max_cross_year = [] # years when index value will be higher than its historical max
                min_cross_year = [] # years when index value will be lower than its historical min

                max_value_for_region_per_year = region_data.max(dim='x').max(dim='y')
                min_value_for_region_per_year = region_data.min(dim='x').min(dim='y')
                
                
                for i in range(85):
                    if max_value_for_region_per_year[i] > max_min_results[region][index][model]["max"]:
                        max_cross_year.append(2015+i)
                    if min_value_for_region_per_year[i] < max_min_results[region][index][model]["min"]:
                        min_cross_year.append(2015+i)
                
                max_min_results[region][index][model]["max_cross_years"] = max_cross_year
                max_min_results[region][index][model]["min_cross_years"] = min_cross_year
                
                max_count = 1
                max_consi_yearyear= None
                
                for i in range(len(max_cross_year)):
                    if(i==0):
                        continue
                    else:
                        if(max_cross_year[i] == max_cross_year[i-1]+1):
                            if(max_consi_year == None):
                                max_consi_year = max_cross_year[i-1]
                            max_count= max_count+1
                            if(max_count>=5):
                                break
                        else:
                            if(max_count>=5):
                                break
                            else:
                                max_count=1;
                                max_consi_year = None
                                
                    if (i == (len(max_cross_year)-1)) & (max_count<5):
                        max_consi_year = None;
                        
                
                min_count = 1
                min_consi_year= None
                
                for j in range(len(min_cross_year)):
                    if(j == 0):
                        continue
                    else:
                        if(min_cross_year[j] == min_cross_year[j-1]+1):
                            if(min_consi_year == None):
                                min_consi_year = min_cross_year[j-1]
                            min_count= min_count+1
                            if(min_count>=5):
                                break
                        else:
                            if(min_count>=5):
                                break
                            else:
                                min_count=1;
                                min_consi_year = None
                    if (j == (len(max_cross_year)-1)) & (max_count<5):
                        max_consi_year = None;
                
                max_min_results[region][index][model]["max_cross_consi"] = max_consi_year
                max_min_results[region][index][model]["min_cross_consi"] = min_consi_year
                

Precssing of IIT-ESM (INDIA) is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [02:09<00:00, 10.78s/it]


## Summary of Max & min values in history and exposure time of future of different models

In [7]:
summary_results = []

for region in regions:
    for index in indices:
        for model in models:
            #numpy_array = max_min_results[region][model][index]["max"].to_numpy()
            max_vals = max_min_results[region][index][model]["max"].values
            min_vals = max_min_results[region][index][model]["min"].values

            max_ = np.nanmedian(max_vals)
            min_ = np.nanmedian(min_vals)
            
            summary_results.append({
                "Region": region,
                "Index": index,
                "Model": model,
                "Max": max_,
                "Min": min_,
                "Max_Cross_start": max_min_results[region][index][model]["max_cross_consi"],
                "Min_Cross_start": max_min_results[region][index][model]["min_cross_consi"],
                "Max_Cross_Years": max_min_results[region][index][model]["max_cross_years"],
                "Min_Cross_Years": max_min_results[region][index][model]["min_cross_years"],
            })

summary_df = pd.DataFrame(summary_results)

Styling the dataframe

In [8]:
styled_df = summary_df.style.set_properties(**{'text-align': 'center'})
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('text-align', 'right')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('font-weight', 'bold')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('font-weight', 'normal')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('background-color', '#f2f2f2')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('background-color', '#e6e6e6')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('border-bottom', '1px solid #ddd')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('border-bottom', '1px solid #ddd')])])
styled_df = styled_df.set_caption('Abrupt Values')
styled_df = styled_df.format({ "Region": "{:.0f}",  "Max": "{:.2f}", "Min": "{:.2f}" })
styled_df = styled_df.hide(axis='index')
styled_df = styled_df.background_gradient(cmap='coolwarm')

#styled_df = styled_df.bar(subset=['Max', 'Min'])

# To display the styled DataFrame
styled_df

Region,Index,Model,Max,Min,Max_Cross_start,Min_Cross_start,Max_Cross_Years,Min_Cross_Years
0,AT_in,IIT-ESM (INDIA),32.59,-23.47,2037.000000,nan,"[2020, 2026, 2030, 2031, 2033, 2034, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,AT_out,IIT-ESM (INDIA),31.77,-31.35,2037.000000,nan,"[2020, 2023, 2026, 2030, 2033, 2034, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,DI,IIT-ESM (INDIA),27.26,-24.24,2037.000000,nan,"[2020, 2026, 2030, 2031, 2033, 2034, 2035, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,ET,IIT-ESM (INDIA),23.55,-11.34,2037.000000,nan,"[2030, 2031, 2034, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,HI,IIT-ESM (INDIA),-84.87,-382.16,2042.000000,2053.000000,"[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]","[2017, 2021, 2025, 2026, 2027, 2029, 2033, 2036, 2040, 2042, 2044, 2046, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2065, 2066, 2068, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]"
0,HMI,IIT-ESM (INDIA),26.53,-29.90,2037.000000,nan,"[2030, 2031, 2034, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,MDI,IIT-ESM (INDIA),29.11,-27.35,2037.000000,nan,"[2017, 2020, 2024, 2026, 2029, 2030, 2031, 2033, 2034, 2035, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099]",[]
0,NET,IIT-ESM (INDIA),25.18,-43.80,2037.000000,nan,"[2020, 2026, 2030, 2031, 2033, 2034, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2

In [9]:
styled_df.to_html('../Processed_Data/Output of analysis/max_min_values.html')

In [10]:
styled_df.to_excel('../Processed_Data/Output of analysis/max_min_values.xlsx')

In [11]:
summary_df.to_csv('../Processed_Data/Output of analysis/max_min_values.csv')

## Finding one value of time median

In [11]:
final_results = {region: {index: {"median_max_cross_consi": None, "median_min_cross_consi": None} for index in indices}  for region in regions}

for region in regions:
    for index in indices:
        max_list = []
        min_list = []
        
        for model in models:
            if max_min_results[region][index][model]['max_cross_consi']:
                max_list.append(max_min_results[region][index][model]['max_cross_consi'])
            if max_min_results[region][index][model]['min_cross_consi']:
                min_list.append(max_min_results[region][index][model]['min_cross_consi'])
        
        if len(max_list)!=0:
            final_results[region][index]['median_max_cross_consi'] = statistics.median(max_list)
        if len(min_list)!=0:
            final_results[region][index]['median_min_cross_consi'] = statistics.median(min_list)
        

In [12]:
summary_median= []

for region in regions:
    for index in indices:
            
        summary_median.append({
            "Region": region,
            "Index": index,
            "Max_Median": final_results[region][index]['median_max_cross_consi'],
            "Min_Median": final_results[region][index]['median_min_cross_consi']
        })


summary_median_df = pd.DataFrame(summary_median)

In [13]:
styled_median_df = summary_median_df.style.set_properties(**{'text-align': 'center'})
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('text-align', 'right')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('font-weight', 'bold')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('font-weight', 'normal')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('background-color', '#f2f2f2')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('background-color', '#e6e6e6')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('border-bottom', '1px solid #ddd')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('border-bottom', '1px solid #ddd')])])
styled_median_df = styled_median_df.set_caption('Abrupt Values')
styled_median_df = styled_median_df.format({ "Region": "{:.0f}",  "Max_Median": "{:.2f}", "Min_Median": "{:.2f}" })
styled_median_df = styled_median_df.hide(axis='index')
styled_median_df = styled_median_df.background_gradient(cmap='coolwarm')

#styled_df = styled_df.bar(subset=['Max_Median', 'Min_Median'])

# To display the styled DataFrame
styled_median_df

Region,Index,Max_Median,Min_Median
0,AT_in,2037.00,nan
0,AT_out,2037.00,nan
0,DI,2037.00,nan
0,ET,2037.00,nan
0,HI,2042.00,2053.00
0,HMI,2037.00,nan
0,MDI,2037.00,nan
0,NET,2037.00,nan
0,SAT,2037.00,nan
0,sWGBT,2037.00,nan


In [14]:
styled_median_df.to_excel('../Processed_Data/Output of analysis/median_values_of_exposure_timing.xlsx')

In [17]:
summary_median_df.to_csv('../Processed_Data/Output of analysis/median_values_of_exposure_timing.csv')